<a href="https://colab.research.google.com/github/rajeevfromkrec/NLP/blob/master/text%20processing%20and%20prediction%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

--2020-07-25 16:09:14--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  1.22MB/s    in 2.5s    

2020-07-25 16:09:17 (1.22 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



In [2]:
!tar -xzf review_polarity.tar.gz

In [3]:
!ls

poldata.README.2.0  review_polarity.tar.gz  sample_data  txt_sentoken


In [40]:
import string
import re
from os import listdir
from nltk.corpus import stopwords
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [41]:
def load_doc(filename): 
# open the file as read only 
  file = open(filename, 'r') 
# read all text 
  text = file.read() 
# close the file 
  file.close()
  return text
  

In [42]:
def clean_doc(doc):
  tokens=doc.split()
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  tokens = [re_punc.sub('', w) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
  tokens = [word for word in tokens if len(word) > 1] 
  return tokens         

In [43]:
def doc_to_lines(filename, vocab):
  doc=load_doc(filename)
  tokens=clean_doc(doc)
  tokens=[ w for w in tokens if w in vocab]
  return ' '.join(tokens)

In [47]:
def save_list(lines, filename): 
# convert lines to a single blob of text 
  data = '\n'.join(lines) 
  # open file 
  file = open(filename, 'w') 
  # write text 
  file.write(data) 
  # close file 
  file.close()

In [52]:
# define vocab 
vocab = Counter() 
# add all docs to vocab 
process_docs('txt_sentoken/pos', vocab, 0) 
process_docs('txt_sentoken/neg', vocab, 0) 
# print the size of the vocab 
print(len(vocab)) 
# keep tokens with a min occurrence min_occurane = 2 
tokens = [k for k,c in vocab.items() if c >= min_occurane] 
print(len(tokens)) 
# save tokens to a vocabulary file 
save_list(tokens, 'vocab.txt')


0
0


In [53]:
!cat vocab.txt

In [45]:
|def process_docs(directory,vocab, is_train):
  lines= list()
  for filename in listdir(directory):
    if is_train and filename.startswith('CV9'):
      continue
    if not is_train and not filename.startswith('cv9'):
      continue
    path= directory + '/'+filename
    line= doc_to_lines(path, vocab)
    lines.append(line)
    return lines


In [13]:
def load_clean_dataset(is_train,vocab):
  neg= process_docs('txt_sentoken/neg', vocab, is_train)
  pos= process_docs('txt_sentoken/pos', vocab, is_train)

  docs=neg+pos

  labels = [0 for _ in range(len(neg))] +[1 for _ in range(len(pos))]
  return docs , labels
  

In [25]:
def define_model(n_words):
  model= Sequential()
  model.add(Dense(50,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.comple(loss='binary_crossentropy',optimizer='ADAM',metrics=['accuracy'])
  model.summary()
  return model


In [17]:
def create_tokenizer(lines): 
  tokenizer = Tokenizer() 
  tokenizer.fit_on_texts(lines) 
  return tokenize

In [26]:
def evaluate_mode(Xtrain, ytrain, Xtest, ytest):
  scores= list()
  n_repeat=10
  n_words=Xtest.shape[1]
  for i in range(n_repeat):
    model=define_model(n_words)
    model.fit(Xtrain,ytrain,epochs=10, verbose=0)
    _, acc=model.evaluate(Xtest,ytest,verbose=0)
    scores.append(acc)
    print('%d accuracy: %s' % ((i+1), acc))
    return scores

def predict_sentiment(review,vocab,tokenizer,model):
  tokens=clean_doc(review)
  tokens = [w for w in tokens if w in vocab]
  line = ' '.join(tokens)
  encoded = tokenizer.texts_to_matrix([line], mode='binary')
  yhat = model.predict(encoded, verbose=0)
  percent_pos = yhat[0,0]
  if round(percent_pos) == 0: 
    return (1-percent_pos), 'NEGATIVE'
  return percent_pos, 'POSITIVE'
